In this notebook we will deomonstrate aspect based sentiment analysis using [Varder](https://github.com/cjhutto/vaderSentiment) and [Stanford Core NLP](https://stanfordnlp.github.io/CoreNLP/index.html).<br>
<br>**VADER Sentiment Analysis**: VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.(source:[github](https://github.com/cjhutto/vaderSentiment))<br>
Stanford NLP have a live demo of aspect based sentiment analysis [here](http://nlp.stanford.edu:8080/sentiment/rntnDemo.html).<br><br>
**Stanford Core NLP**: "Most sentiment prediction systems work just by looking at words in isolation, giving positive points for positive words and negative points for negative words and then summing up these points. That way, the order of words is ignored and important information is lost. In constrast, our new deep learning model actually builds up a representation of whole sentences based on the sentence structure. It computes the sentiment based on how words compose the meaning of longer phrases. This way, the model is not as easily fooled as previous models."(source: [Stanford Core NLP](https://nlp.stanford.edu/sentiment/index.html).)

In [ ]:
!pip install vaderSentiment
!pip install pycorenlp

### Importing the necessary packages

In [2]:
from pprint import pprint

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string

import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize, RegexpTokenizer

from pycorenlp import StanfordCoreNLP

[nltk_data] Downloading package punkt to /home/samyak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/samyak/nltk_data...


Lets analyze these three sentences. 

In [3]:
positive = "This fried chicken tastes very good. It is juicy and perfectly cooked."
negative = "This fried chicken tasted bad. It is dry and overcooked."
ambiguous = "Except the amazing fried chicken everything else at the restaurant tastes very bad."

### VarderSentiment
It scores from -1 to 1. -1 being negative and 1 being positive

In [4]:
def sentiment_analyzer_scores(text):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    score = sentiment_analyzer.polarity_scores(text)
    pprint(text)
    pprint(score)
    print("-"*30)

In [5]:
print("Positive:")
sentiment_analyzer_scores(positive)

print("Negative:")
sentiment_analyzer_scores(negative)

print("Ambiguous:")
sentiment_analyzer_scores(ambiguous)

Positive:
'This fried chicken tastes very good. It is juicy and perfectly cooked.'
{'compound': 0.8122, 'neg': 0.0, 'neu': 0.575, 'pos': 0.425}
------------------------------
Negative:
'This fried chicken tasted bad. It is dry and overcooked.'
{'compound': -0.5423, 'neg': 0.28, 'neu': 0.72, 'pos': 0.0}
------------------------------
Ambiguous:
('Except the amazing fried chicken everything else at the restaurant tastes '
 'very bad.')
{'compound': 0.0018, 'neg': 0.204, 'neu': 0.592, 'pos': 0.204}
------------------------------


As expected the sentiment analyzer performed well on the positive and negative case. When taking into consideration the ambiguous sentence, it calculated the compound sentiment to be close to 0, i.e, neutral.<br>
But it seems to be a negative comment.

In [8]:
def get_word_sentiment(text):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    
    tokenized_text = nltk.word_tokenize(text)
    
    positive_words=[]
    neutral_words=[]
    negative_words=[]
    for word in tokenized_text:
        if (sentiment_analyzer.polarity_scores(word)['compound']) >= 0.1:
            positive_words.append(word)
        elif (sentiment_analyzer.polarity_scores(word)['compound']) <= -0.1:
            negative_words.append(word)
        else:
            neutral_words.append(word)
    print(text)
    print('Positive:',positive_words)        
    print('Negative:',negative_words)    
    print('Neutral:',neutral_words)
    print("-"*30)

In [9]:
get_word_sentiment(positive)
get_word_sentiment(negative)
get_word_sentiment(ambiguous)

This fried chicken tastes very good. It is juicy and perfectly cooked.
Positive: ['good', 'perfectly']
Negative: []
Neutral: ['This', 'fried', 'chicken', 'tastes', 'very', '.', 'It', 'is', 'juicy', 'and', 'cooked', '.']
------------------------------
This fried chicken tasted bad. It is dry and overcooked.
Positive: []
Negative: ['bad']
Neutral: ['This', 'fried', 'chicken', 'tasted', '.', 'It', 'is', 'dry', 'and', 'overcooked', '.']
------------------------------
Except the amazing fried chicken everything else at the restaurant tastes very bad.
Positive: ['amazing']
Negative: ['bad']
Neutral: ['Except', 'the', 'fried', 'chicken', 'everything', 'else', 'at', 'the', 'restaurant', 'tastes', 'very', '.']
------------------------------


### Stanford Core NLP
Before moving on to execute the code we need to start the Stanford Core NLP server on our local machine.<br> To do that follow the steps below (tested on debian should work fine for other distributions too):
1. Download the Stanford Core NLP model from [here](https://stanfordnlp.github.io/CoreNLP/#download).
2. Unizip the folder
3. cd into the folder<br>
    ```cd stanford-corenlp-4.0.0/```
4. Start the server using this command:<br>
    ```java -mx5g -cp "./*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000```
<br><br>
If you do not have java installed on your system please install it from the official [Oracle](https://www.oracle.com/in/java/technologies/javase-downloads.html) page.
<br><br>


In [16]:
nlp = StanfordCoreNLP('http://localhost:9000')

def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 1000,
                       })
    print(text)
    print('Sentiment:', res['sentences'][0]['sentiment'])
    print('Sentiment score:', res['sentences'][0]['sentimentValue'])
    print('Sentiment distribution (0-v. negative, 5-v. positive:', res['sentences'][0]['sentimentDistribution'])
    print("-"*30)

In [17]:
get_sentiment(positive)
get_sentiment(negative)
get_sentiment(ambiguous)

This fried chicken tastes very good. It is juicy and perfectly cooked.
Sentiment: Negative
Sentiment score: 1
Sentiment distribution (0-v. negative, 5-v. positive: [0.14784497645863, 0.42150440150006, 0.26795232397677, 0.14973929036748, 0.01295900769705]
------------------------------
This fried chicken tasted bad. It is dry and overcooked.
Sentiment: Verynegative
Sentiment score: 0
Sentiment distribution (0-v. negative, 5-v. positive: [0.59929214372637, 0.37183747332338, 0.00491027284258, 0.01321244183821, 0.01074766826945]
------------------------------
Except the amazing fried chicken everything else at the restaurant tastes very bad.
Sentiment: Negative
Sentiment score: 1
Sentiment distribution (0-v. negative, 5-v. positive: [0.14730433605028, 0.42068997423614, 0.26920777577949, 0.14985314657637, 0.01294476735772]
------------------------------


Here you see the model successfully predicts the ambigous sentence which the Varder failed to predict correctly.<br>
The code in this notebook has been adapted from this [article](https://towardsdatascience.com/sentiment-analysis-beyond-words-6ca17a6c1b54).See below code for colab.